In [1]:
!pip install -q cleanlab

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import cleanlab
import numpy as np
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
# To silence convergence warnings caused by using a weak
# logistic regression classifier on image data
# import warnings
# warnings.simplefilter("ignore")
np.random.seed(477)

### 単純なチュートリアル, ただしnumpyで実装されている

#### 入力
s ... ノイジーラベルの行列 shape=(n,1) 要素はint型である必要がありそう。

psx ... oofの確率行列 shape=(n_samples,m_targets) 

#### 出力
confident_joint ... shape=(m,m),推定された同時分布

label_errors_idx ... 推定されたlabel errorのidxたち

In [3]:
# まずはモデルを訓練してsとpsxを得る
# Get handwritten digits data
X = load_digits()['data']
y = load_digits()['target']
print('Handwritten digits datasets number of classes:', len(np.unique(y)))
print('Handwritten digits datasets number of examples:', len(y), y)

# Add lots of errors to labels
NUM_ERRORS = 100
s = np.array(y)
#100個mislabelにするサンプルを選ぶ
error_indices = np.random.choice(len(s), NUM_ERRORS, replace=False)
for i in error_indices:
    # Switch to some wrong label thats a different class
    wrong_label = np.random.choice(np.delete(range(10), s[i]))
    s[i] = wrong_label

# Confirm that we indeed added NUM_ERRORS label errors
assert (len(s) - sum(s == y) == NUM_ERRORS)
actual_label_errors = np.arange(len(y))[s != y]
print('\nIndices of actual label errors:\n', actual_label_errors)

# To keep the tutorial short, we use cleanlab to get the 
# out-of-sample predicted probabilities using cross-validation
# with a very simple, non-optimized logistic regression classifier
# cross validateもする
psx = cleanlab.latent_estimation.estimate_cv_predicted_probabilities(
    X,
    s,
    clf=LogisticRegression(max_iter=1000, multi_class='auto', solver='lbfgs'))
print('shape of psx',psx.shape)

Handwritten digits datasets number of classes: 10
Handwritten digits datasets number of examples: 1797 [0 1 2 ... 8 9 8]

Indices of actual label errors:
 [  24   27   42   47   50   54   58   62   85   94  105  107  110  157
  159  182  201  211  220  223  232  300  310  312  336  343  348  350
  358  366  375  387  400  418  430  462  469  483  502  538  555  559
  562  564  614  623  626  647  649  674  694  706  708  761  778  796
  807  830  834  840  883  887  980  996 1007 1044 1064 1083 1084 1104
 1105 1111 1122 1176 1192 1257 1261 1265 1301 1312 1324 1327 1341 1375
 1472 1483 1492 1493 1496 1524 1534 1584 1592 1643 1646 1648 1675 1707
 1744 1779]


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


shape of psx (1797, 10)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Qを推定する
まさかのフルスクラッチ...

In [4]:
# Find the number of unique classes if K is not given
K = len(np.unique(s))

# Estimate the probability thresholds for confident counting
# You can specify these thresholds yourself if you want
# as you may want to optimize them using a validation set.
# By default (and provably so) they are set to the average class prob.
thresholds = [np.mean(psx[:,k][s == k]) for k in range(K)] # P(s^=k|s=k)
thresholds = np.asarray(thresholds)

# Compute confident joint
confident_joint = np.zeros((K, K), dtype = int)
# Countは自前実装なのか...
for i, row in enumerate(psx):
    s_label = s[i]
    # Find out how many classes each example is confidently labeled as
    confident_bins = row >= thresholds - 1e-6
    num_confident_bins = sum(confident_bins)
    # If more than one conf class, inc the count of the max prob class
    if num_confident_bins == 1:
        confident_joint[s_label][np.argmax(confident_bins)] += 1
    elif num_confident_bins > 1:
        confident_joint[s_label][np.argmax(row)] += 1

# Normalize confident joint (use cleanlab, trust me on this)
confident_joint = cleanlab.latent_estimation.calibrate_confident_joint(
    confident_joint, s)

cleanlab.util.print_joint_matrix(confident_joint)


 Joint Label Noise Distribution Matrix P(s,y) of shape (10, 10)
 p(s,y)	y=0	y=1	y=2	y=3	y=4	y=5	y=6	y=7	y=8	y=9
	---	---	---	---	---	---	---	---	---	---
s=0 |	170	2	2	0	1	4	0	0	2	0
s=1 |	2	166	3	1	3	1	0	0	3	1
s=2 |	0	3	157	4	0	1	0	1	0	1
s=3 |	0	0	3	176	0	0	1	1	3	1
s=4 |	1	1	0	0	165	1	0	1	2	3
s=5 |	0	2	5	1	4	165	2	0	1	6
s=6 |	0	4	1	1	2	0	167	3	1	5
s=7 |	0	1	0	0	4	3	3	163	0	1
s=8 |	0	3	4	1	4	1	3	4	164	2
s=9 |	1	1	1	3	5	1	2	5	4	156
	Trace(matrix) = 1649



ラベルエラーを見つける
これもまさかのフルスクラッチ

In [5]:
MIN_NUM_PER_CLASS = 5
# Leave at least MIN_NUM_PER_CLASS examples per class.
# NOTE prune_count_matrix is transposed (relative to confident_joint)
prune_count_matrix = cleanlab.pruning.keep_at_least_n_per_class(
    prune_count_matrix=confident_joint.T,
    n=MIN_NUM_PER_CLASS,
)

s_counts = np.bincount(s)
noise_masks_per_class = []
# For each row in the transposed confident joint
for k in range(K):
    noise_mask = np.zeros(len(psx), dtype=bool)
    psx_k = psx[:, k]
    if s_counts[k] > MIN_NUM_PER_CLASS:  # Don't prune if not MIN_NUM_PER_CLASS
        for j in range(K):  # noisy label index (k is the true label index)
            if k != j:  # Only prune for noise rates, not diagonal entries
                num2prune = prune_count_matrix[k][j]
                if num2prune > 0:
                    # num2prune'th largest p(classk) - p(class j)
                    # for x with noisy label j
                    margin = psx_k - psx[:, j]
                    s_filter = s == j
                    threshold = -np.partition(
                        -margin[s_filter], num2prune - 1
                    )[num2prune - 1]
                    noise_mask = noise_mask | (s_filter & (margin >= threshold))
        noise_masks_per_class.append(noise_mask)
    else:
        noise_masks_per_class.append(np.zeros(len(s), dtype=bool))

# Boolean label error mask
label_errors_bool = np.stack(noise_masks_per_class).any(axis=0)

 # Remove label errors if given label == model prediction
for i, pred_label in enumerate(psx.argmax(axis=1)):
    # np.all let's this work for multi_label and single label
    if label_errors_bool[i] and np.all(pred_label == s[i]):
        label_errors_bool[i] = False

# Convert boolean mask to an ordered list of indices for label errors
label_errors_idx = np.arange(len(s))[label_errors_bool]
# self confidence is the holdout probability that an example
# belongs to its given class label
self_confidence = np.array(
    [np.mean(psx[i][s[i]]) for i in label_errors_idx]
)
margin = self_confidence - psx[label_errors_bool].max(axis=1)
label_errors_idx = label_errors_idx[np.argsort(margin)]

print('Indices of label errors found by confident learning:')
print('Note label errors are sorted by likelihood of being an error')
print('but here we just sort them by index for comparison with above.')
print(np.array(sorted(label_errors_idx)))

Indices of label errors found by confident learning:
Note label errors are sorted by likelihood of being an error
but here we just sort them by index for comparison with above.
[   5    9   24   42   43   46   47   50   54   58   62   69   77   85
   94   95  105  107  110  123  157  159  182  191  201  220  223  227
  232  267  300  310  312  325  336  343  348  350  358  361  366  375
  384  387  418  421  430  462  483  492  502  521  538  547  555  559
  562  614  623  626  647  649  674  694  706  708  745  757  761  769
  770  778  794  795  807  830  834  840  883  887  920  980  996 1007
 1022 1025 1038 1044 1064 1083 1084 1104 1105 1111 1118 1122 1146 1176
 1192 1197 1233 1241 1257 1261 1264 1265 1301 1312 1324 1327 1341 1375
 1400 1472 1483 1492 1493 1496 1524 1534 1553 1572 1575 1580 1584 1592
 1597 1615 1618 1643 1645 1646 1648 1658 1660 1675 1707 1742 1744 1747
 1779]


In [6]:
score = sum([e in label_errors_idx for e in actual_label_errors]) / NUM_ERRORS
print('% actual errors that confident learning found: {:.0%}'.format(score))
score = sum([e in actual_label_errors for e in label_errors_idx]) / len(label_errors_idx)
print('% confident learning errors that are actual errors: {:.0%}'.format(score))

% actual errors that confident learning found: 94%
% confident learning errors that are actual errors: 67%
